In [27]:
import numpy as np
import torch
import skorch
import pandas as pd
from torch import nn
from torch.utils.data import DataLoader
from torch.nn.functional import elu,relu,leaky_relu
from torchvision.transforms import Resize
import braindecode 
from braindecode.models import ShallowFBCSPNet,Deep4Net,TCN,ATCNet,HybridNet,EEGConformer
from braindecode.models.modules import Expression
from braindecode.models.functions import squeeze_final_output,safe_log
from skorch.callbacks import Checkpoint
from skorch.helper import predefined_split
from config_exp3 import *
from dataset import *
from sklearn.metrics import roc_auc_score,f1_score
from mne import set_log_level
from skorch.callbacks import LRScheduler
import os
device = 'cuda' if cuda else 'cpu'
ab_label_dict = ['Normal','Sharp Wave','Delta Slow Wave', 'Spike and Wave Discharge', 'Beta Wave', 'Theta Wave', 'Triphasic Wave', 'Low Voltage','Burst Suppression','Unknown']
n_chans=2
input_time_length=input_time_length//n_chans

In [12]:
folders=['Abnormality_Crops','Normal_Crops']
file_names=[]
labels=[]
ab_labels=[]
for fold in folders:
    folder_path=os.path.join(processed_folder,fold)
    files=os.listdir(folder_path)
    for file in files:
        file_path=os.path.join(folder_path,file)
        file_names.append(file_path)
        data=np.load(file_path)
        labels.append(int(data['label']))
        ab_labels.append(int(data['ab_label']))

In [32]:
from sklearn.model_selection import train_test_split
windows=np.array(list(zip(file_names,labels,ab_labels)))
train_data,test_data=train_test_split(windows,test_size=0.2)
train_dataframe=pd.DataFrame({'file_path':train_data[:,0],'label':train_data[:,1],'ab_label':train_data[:,2]})
train_dataframe.to_excel(f"{processed_folder}/train.xlsx",index=False)
test_dataframe=pd.DataFrame({'file_path':test_data[:,0],'label':test_data[:,1],'ab_label':test_data[:,2]})
test_dataframe.to_excel(f"{processed_folder}/eval.xlsx",index=False)

In [2]:
train_dataframe=pd.read_excel(f'{processed_folder}/train.xlsx')
eval_dataframe=pd.read_excel(f'{processed_folder}/eval.xlsx')

train_data=[]
train_label=[]
for i in range(len(train_dataframe)):
    file=np.load(train_dataframe['file_path'][i])
    train_data.append(file['data'])
    train_label.append(train_dataframe['ab_label'][i])
train_data=np.array(train_data)
train_label=np.array(train_label)


eval_data=[]
eval_label=[]
for i in range(len(eval_dataframe)):
    file=np.load(eval_dataframe['file_path'][i])
    eval_data.append(file['data'])
    eval_label.append(eval_dataframe['ab_label'][i])
eval_data=np.array(eval_data)
eval_label=np.array(eval_label)

In [3]:
train_data=train_data.reshape(len(train_data),n_chans,input_time_length,order='F')
eval_data=eval_data.reshape(len(eval_data),n_chans,input_time_length,order='F')

In [5]:
train_set=skorch.dataset.Dataset(train_data,train_label)
test_set=skorch.dataset.Dataset(eval_data,eval_label)

In [10]:
#Make it reshape it as well
#train_x=train_x.reshape(len(train_x),n_chans,input_time_length,order='F')
#This is not giving results, just load all the windows as a single numpy array and train on that.
class Exp3Dataset(torch.utils.data.Dataset):
    def __init__(self, excel_path):
        super().__init__()
        excel_file=pd.read_excel(excel_path)
        self.file_names=excel_file['file_path'].to_numpy(dtype=str)
        self.label=excel_file['label'].to_numpy()
        self.ab_label=excel_file['ab_label'].to_numpy()
    def __getitem__(self, index):
        file=np.load(self.file_names[index])
        window=file['data']
        window=window.reshape(n_chans,input_time_length,order='F')
        #window=np.expand_dims(window,axis=-1)
        #print(window.shape)
        ab_label=np.array(self.ab_label[index])
        #ab_label=np.eye(len(ab_label_dict), dtype='uint8')[ab_label]
        return window,ab_label
 
    def __len__(self):
        return len(self.file_names)

In [11]:
test_set=Exp3Dataset(f'{processed_folder}/eval.xlsx')
train_set=Exp3Dataset(f'{processed_folder}/train.xlsx')

In [12]:
train_set.__getitem__(0)[0].shape

(2, 300)

In [13]:
train_set.__getitem__(0)[1]

array(5, dtype=int64)

In [14]:
model_name="shallow_deep"

In [18]:
criterion=torch.nn.NLLLoss
n_classes = len(ab_label_dict)
if model_name=="shallow":
    optimizer_lr = 0.0000625
    optimizer_weight_decay = 0
    pool_time_length = 16
    pool_time_stride = 2
    filter_time_length = 16
    n_filters_spat = 64
    n_filters_time = 32
    split_first_layer = True
    drop_prob = 0.328794
    #The final conv length is auto to ensure that output will give two values for single EEG window
    model = ShallowFBCSPNet(n_chans,
                                    n_classes,
                                    #n_filters_time=n_start_chans,
                                    #n_filters_spat=n_start_chans,
                                    n_times=input_time_length,
                                    pool_time_length=pool_time_length,
                                    pool_time_stride=pool_time_stride,
                                    drop_prob=drop_prob,
                                    n_filters_time=n_filters_time,
                                    n_filters_spat=n_filters_spat,
                                    filter_time_length=filter_time_length,
                                    split_first_layer=split_first_layer,
                                    final_conv_length='auto',)
    test=torch.ones(size=(7,n_chans,input_time_length))
    out=model.forward(test)
    print(out.shape)
elif model_name=="deep":
    optimizer_lr = init_lr
    optimizer_weight_decay = 0
    model = Deep4Net(n_chans, n_classes,
                         n_filters_time=n_start_chans,
                         n_filters_spat=n_start_chans,
                         n_times=input_time_length,
                         n_filters_2 = int(n_start_chans * n_chan_factor),
                         n_filters_3 = int(n_start_chans * (n_chan_factor ** 2.0)),
                         n_filters_4 = int(n_start_chans * (n_chan_factor ** 3.0)),
                         final_conv_length='auto',
                        stride_before_pool=True)
    test=torch.ones(size=(6,n_chans,input_time_length))
    out=model.forward(test)
    print(out.shape)
elif model_name=="deep_smac" or model_name == 'deep_smac_bnorm':
    optimizer_lr = 0.0000625
    if model_name == 'deep_smac':
            do_batch_norm = False
    else:
        do_batch_norm = True
    drop_prob = 0.244445
    filter_length_2 = 4
    filter_length_3 = 8
    filter_length_4 = 12
    filter_time_length = 8
    #final_conv_length = 1
    first_nonlin = elu
    first_pool_mode = 'mean'
    later_nonlin = elu
    later_pool_mode = 'mean'
    n_filters_factor = 1.679066
    n_filters_start = 32
    pool_time_length = 1
    pool_time_stride = 1
    split_first_layer = True
    n_chan_factor = n_filters_factor
    n_start_chans = n_filters_start
    model = Deep4Net(n_chans, n_classes,
            n_filters_time=n_start_chans,
            n_filters_spat=n_start_chans,
            n_times=input_time_length,
            n_filters_2=int(n_start_chans * n_chan_factor),
            n_filters_3=int(n_start_chans * (n_chan_factor ** 2.0)),
            n_filters_4=int(n_start_chans * (n_chan_factor ** 3.0)),
            final_conv_length='auto',
            stride_before_pool=True,
            drop_prob=drop_prob,
            filter_length_2=filter_length_2,
            filter_length_3=filter_length_3,
            filter_length_4=filter_length_4,
            filter_time_length=filter_time_length,
            first_conv_nonlin=first_nonlin,
            first_pool_mode=first_pool_mode,
            later_conv_nonlin=later_nonlin,
            later_pool_mode=later_pool_mode,
            pool_time_length=pool_time_length,
            pool_time_stride=pool_time_stride,
            split_first_layer=split_first_layer
            )
    test=torch.ones(size=(6,n_chans,input_time_length))
    out=model.forward(test)
    print(out.shape)
    del do_batch_norm,drop_prob,filter_length_2,filter_length_3,filter_length_4,filter_time_length,first_nonlin,n_chan_factor,n_start_chans,first_pool_mode,later_nonlin,later_pool_mode,n_filters_factor,n_filters_start,pool_time_length,pool_time_stride,split_first_layer
#Works properly, fit the hybrid cnn
elif model_name=="hybrid":
    optimizer_lr = 0.0000625
    optimizer_weight_decay = 0
    #The final conv length is auto to ensure that output will give two values for single EEG window
    model = HybridNet(n_chans, n_classes,n_times=input_time_length,)
    test=torch.ones(size=(2,n_chans,input_time_length))
    out=model.forward(test)
    out_length=out.shape[2]
    model.final_layer=nn.Conv2d(100,n_classes,(out_length,1),bias=True,)
    model=nn.Sequential(model,nn.Flatten(),nn.LogSoftmax(dim=1))
    out=model.forward(test)
    print(out.shape)
    del out_length
elif model_name=="TCN":
    import warnings
    #This disables the warning of the dropout2d layers receiving 3d input
    warnings.filterwarnings("ignore")
    optimizer_lr = 0.0000625
    optimizer_weight_decay = 0
    n_blocks=7
    n_filters=32
    kernel_size=24
    drop_prob = 0.3
    x=TCN(n_chans,n_classes,n_blocks,n_filters,kernel_size,drop_prob)
    test=torch.ones(size=(7,n_chans,input_time_length))
    out=x.forward(test)
    out_length=out.shape[2]
    #There is no hyperparameter where output of TCN is (Batch_Size,Classes) when input is (Batch_Size,21,6000) so add new layers to meet size
    model=nn.Sequential(x,nn.Conv1d(n_classes,n_classes,out_length,bias=True,),nn.LogSoftmax(dim=1),nn.Flatten())
    out=model.forward(test)
    print(out.shape)
    del out_length,x
elif model_name=="shallow_deep":
    drop_prob = 0.244445
    filter_length_2 = 8
    filter_length_3 = 12
    filter_length_4 = 16
    n_filters_factor = 1.679066
    n_filters_start = 32
    split_first_layer = True
    n_chan_factor = n_filters_factor
    #n_start_chans = n_filters_start

    optimizer_lr = 0.0000625
    optimizer_weight_decay = 0
    filter_time_length=4
    first_conv_nonlin=relu
    first_pool_nonlin=safe_log
    later_conv_nonlin=elu
    later_pool_nonlin=safe_log
    first_pool_mode = "mean"
    later_pool_mode = "mean"
    pool_time_length=2
    pool_time_stride=2
    model = Deep4Net(n_chans, n_classes,
                            n_filters_time=n_start_chans,
                            n_filters_spat=n_start_chans,
                            n_times=input_time_length,
                            n_filters_2 = int(n_start_chans * n_chan_factor),
                            n_filters_3 = int(n_start_chans * (n_chan_factor ** 2.0)),
                            n_filters_4 = int(n_start_chans * (n_chan_factor ** 3.0)),
                            final_conv_length='auto',
                            first_pool_nonlin=first_pool_nonlin,
                            first_conv_nonlin=first_conv_nonlin,
                            #later_pool_nonlin=later_pool_nonlin,
                            #later_conv_nonlin=later_conv_nonlin,
                            filter_time_length=filter_time_length,
                            pool_time_length=pool_time_length,
                            pool_time_stride=pool_time_stride,
                            first_pool_mode=first_pool_mode,
                            later_pool_mode=later_pool_mode,
                            split_first_layer=split_first_layer,
                            drop_prob=drop_prob,
                            filter_length_2=filter_length_2,
                            filter_length_3=filter_length_3,
                            filter_length_4=filter_length_4,
                            )
    test=torch.ones(size=(7,n_chans,input_time_length))
#    out=model.forward(test)
#    print(out.shape)

elif model_name=="attention":
    optimizer_lr = 0.0000625
    optimizer_weight_decay = 0
    model=ATCNet(n_chans,n_classes,input_time_length//sampling_freq,sampling_freq,concat=True)
    test=torch.ones(size=(7,n_chans,input_time_length))
    out=model.forward(test)
    print(out.shape)
elif model_name=="transformer":
    optimizer_lr = 0.0000625
    optimizer_weight_decay = 0
    criterion=torch.nn.CrossEntropyLoss
    n_filters_time=20
    att_depth=8
    filter_time_length=35
    att_heads=5
    model=EEGConformer(n_outputs=n_classes,n_chans=n_chans,n_times=input_time_length,input_window_seconds=input_time_length//sampling_freq,
                       sfreq=sampling_freq,final_fc_length=7860,n_filters_time=n_filters_time,att_depth=att_depth,
                       filter_time_length=filter_time_length,att_heads=att_heads)
    test=torch.ones(size=(7,n_chans,input_time_length))
    out=model.forward(test)
    print(out.shape)
if cuda:
    model.cuda()
del test
print(model_name)

shallow_deep


In [19]:
model

Deep4Net(
  (ensuredims): Ensure4d()
  (dimshuffle): Rearrange('batch C T 1 -> batch 1 T C')
  (conv_time_spat): CombinedConv(
    (conv_time): Conv2d(1, 25, kernel_size=(4, 1), stride=(1, 1))
    (conv_spat): Conv2d(25, 25, kernel_size=(1, 2), stride=(1, 1), bias=False)
  )
  (bnorm): BatchNorm2d(25, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_nonlin): Expression(expression=relu) 
  (pool): AvgPool2dWithConv()
  (pool_nonlin): Expression(expression=safe_log) 
  (drop_2): Dropout(p=0.244445, inplace=False)
  (conv_2): Conv2d(25, 41, kernel_size=(8, 1), stride=(1, 1), bias=False)
  (bnorm_2): BatchNorm2d(41, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (nonlin_2): Expression(expression=elu) 
  (pool_2): AvgPool2dWithConv()
  (pool_nonlin_2): Expression(expression=identity) 
  (drop_3): Dropout(p=0.244445, inplace=False)
  (conv_3): Conv2d(41, 70, kernel_size=(12, 1), stride=(1, 1), bias=False)
  (bnorm_3): BatchNorm2d(70, eps=1e-05, momen

In [14]:
loss=nn.NLLLoss()
input=[]
input.append(train_set.__getitem__(0)[0])
input=np.array(input)
input=torch.from_numpy(input).cuda()
target=[]
target.append(train_set.__getitem__(0)[1])
target=np.array(target)
target=torch.from_numpy(target).cuda()

In [16]:
output=loss(model(input),target)
output.backward()

In [20]:
monitor = lambda net: any(net.history[-1, ('valid_accuracy_best','valid_f1_best','valid_loss_best')])
cp=Checkpoint(monitor='valid_f1_best',dirname='model',f_params=f'{model_name}best_param.pkl',
               f_optimizer=f'{model_name}best_opt.pkl', f_history=f'{model_name}best_history.json')
path=f'{model_name}II'
classifier = braindecode.EEGClassifier(
        model,
        criterion=criterion,
        optimizer=torch.optim.AdamW,
        train_split=predefined_split(test_set),
        optimizer__lr=optimizer_lr,
        #optimizer__weight_decay=optimizer_weight_decay,
        iterator_train=DataLoader,
        iterator_valid=DataLoader,
        iterator_train__shuffle=True,
        iterator_train__pin_memory=True,
        iterator_valid__pin_memory=True,
        #iterator_train__num_workers=1,
        #iterator_valid__num_workers=1,
        #iterator_train__persistent_workers=True,
        #iterator_valid__persistent_workers=True,
        batch_size=64,
        device=device,
        callbacks=["accuracy"],#,"f1",cp,],
        warm_start=True,
        )
classifier.initialize()

<class 'braindecode.classifier.EEGClassifier'>[initialized](
  module_=============================================================================================================================================
  Layer (type (var_name):depth-idx)        Input Shape               Output Shape              Param #                   Kernel Shape
  Deep4Net (Deep4Net)                      [1, 2, 300]               [1, 10]                   --                        --
  ├─Ensure4d (ensuredims): 1-1             [1, 2, 300]               [1, 2, 300, 1]            --                        --
  ├─Rearrange (dimshuffle): 1-2            [1, 2, 300, 1]            [1, 1, 300, 2]            --                        --
  ├─CombinedConv (conv_time_spat): 1-3     [1, 1, 300, 2]            [1, 25, 297, 1]           1,375                     --
  ├─BatchNorm2d (bnorm): 1-4               [1, 25, 297, 1]           [1, 25, 297, 1]           50                        --
  ├─Expression (conv_nonlin): 1-5 

In [21]:
test=np.random.rand(3,n_chans,input_time_length)
out=classifier.predict(test)
print(out)

[5 5 5]


In [22]:
test_set.__getitem__(0)[0].shape

(2, 300)

In [23]:
test_set.__getitem__(0)[1]

array(5, dtype=int64)

In [16]:
classifier.fit(train_set,epochs=100)

<class 'braindecode.classifier.EEGClassifier'>[initialized](
  module_=============================================================================================================================================
  Layer (type (var_name):depth-idx)        Input Shape               Output Shape              Param #                   Kernel Shape
  ShallowFBCSPNet (ShallowFBCSPNet)        [1, 10, 60]               [1, 10]                   --                        --
  ├─Ensure4d (ensuredims): 1-1             [1, 10, 60]               [1, 10, 60, 1]            --                        --
  ├─Rearrange (dimshuffle): 1-2            [1, 10, 60, 1]            [1, 1, 60, 10]            --                        --
  ├─CombinedConv (conv_time_spat): 1-3     [1, 1, 60, 10]            [1, 25, 36, 1]            6,900                     --
  ├─BatchNorm2d (bnorm): 1-4               [1, 25, 36, 1]            [1, 25, 36, 1]            50                        --
  ├─Expression (conv_nonlin_exp): 

In [ ]:
classifier.load_params(
        f_params=f'model/{model_name}best_param.pkl', f_history=f'model/{model_name}best_history.json')
print("Paramters Loaded")
pred_labels=classifier.predict(test_set)
actual_labels=[label[1] for label in test_set]

In [33]:
#auc=roc_auc_score(actual_labels,classifier.predict_proba(test_set)[:,1],multi_class='ovr')
actual_labels=np.array(actual_labels)
accuracy=np.mean(pred_labels==actual_labels)
f1=f1_score(actual_labels,pred_labels, average='weighted')
print(model_name)
#Accuracy seems incorrect compared to the one reported by skorch
print(f"Accuracy:{accuracy}")
print(f"F1-Score:{f1}")
#print(f"roc_auc score:{auc}")

shallow
Accuracy:0.5634095634095634
F1-Score:0.521509870819439
